In [1]:
import os
import mat73
import math
import pandas as pd
import numpy as np
from scipy.io import savemat
import xgboost as xgb
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, roc_curve, auc, f1_score, accuracy_score, balanced_accuracy_score, cohen_kappa_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.base import clone
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn.utils import class_weight
from sklearn import tree
import matplotlib.pyplot as plt
%matplotlib qt
import seaborn as sns
from itertools import cycle
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
from collections import Counter
import xlsxwriter
import graphviz
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
import random
import time
from matplotlib.lines import Line2D

In [2]:
def IntraIndividualBuilder(df, mm, patient_leftout):
    patient_id = patient_leftout - 1
    first_fallen = int(sum(mm[0 : patient_id]))
    last_fallen = int(first_fallen + mm[patient_id])
    Xytest = df.iloc[first_fallen : last_fallen, :]
    Xytrain = df.drop(df.index[first_fallen : last_fallen], axis = 0)
    return Xytest, Xytrain;

def OnlySomeBuilder(df, mm, patients_array):
    Train = pd.DataFrame()
    summ = 0
    mm_train = np.zeros((1, len(patients_array)))
    for i in range(len(patients_array)):
        ii = patients_array[i]
        Test_i, _ = IntraIndividualBuilder(df, mm, ii)
        Train = pd.concat([Train, Test_i], axis = 0)
        summ += mm[ii]
        mm_train[0, i] = int(mm[ii])
        if Train.shape[0] != int(summ):
            raise NameError('Error in dimensions')
    return Train, mm_train;

def SequentialLearner(df, mm, patient_leftout):
    model = xgb.XGBClassifier(tree_method = 'exact',
                              learning_rate = 0.01,
                              objective = 'multi:softmax',
                              use_label_encoder = False,
                              eval_metric = ['mlogloss', 'auc'],
                              verbosity = 0,
                              max_depth = 10,
                            )
    for i in range(len(mm)):
        if i != patient_leftout:
            test, _ = IntraIndividualBuilder(df, mm, i)
            print('Iteration:', i + 1)
        else:
            print('Skip patient')
        sample_weights = class_weight.compute_sample_weight(class_weight='balanced',
                                                            y = test.iloc[:, -1]
                                                           )
        model.fit(test.drop(columns = ['Sleep Stages']), test.iloc[:, -1], sample_weight = sample_weights)
    return model;

def OrdinalTrain(model_container, features, cardinal_labels, smote_options):
    header_entries = list(cardinal_labels.columns.values)
    for k in range(len(header_entries)):
        cik = header_entries[k]
        label_k = cardinal_3labels[cik]
        
        if smote_options == True:
            sm = SMOTE(random_state = 42)
            Xsm, label_sm = sm.fit_resample(features, label_k)
            model_container[k].fit(Xsm, label_sm)
        else:
            sample_weights = class_weight.compute_sample_weight(class_weight = 'balanced',
                                                                y = label_k,
                                                               )
            model_container[k].fit(features, label_k, sample_weight = sample_weights)
    return model_container;

def First2Hours(train_df):
    last_index = 2 * 60 * 2 - 1
    Xytrain = train_df.iloc[0 : last_index, :]
    Xytest = train_df.drop(train_df.index[0 : last_index], axis = 0)
    return Xytrain, Xytest;

def Evaluation(predictions, n_class):
    prob_array = np.zeros((n_class,))
    epochs = len(predictions)
    counted_labels = Counter(predictions)
    for key in counted_labels:
        if key == 0.0:
            prob_array[0] = counted_labels[key] / epochs
        elif key == 1.0:
            prob_array[1] = counted_labels[key] / epochs
        elif key == 2.0:
            prob_array[2] = counted_labels[key] / epochs
    return prob_array;

def OSAEvaluation(predictions, truelabels):
    truth_array = np.zeros((truelabels.size,))
    for i in range(truelabels.size):
        if truelabels[i] == 0:
            if predictions[i] == 0:
                truth_array[i] = 1
        elif (truelabels[i] == 1) or (truelabels[i] == 2):
            if (predictions[i] == 1) or (predictions[i] == 2):
                truth_array[i] = 1
    return truth_array;

In [3]:
path = 'c:/Users/Jacopo/Desktop/Paper - EDA Sleep Staging/Code Matlab/Keep it Simple/'
os.chdir(path)

DataOSA = mat73.loadmat('FinalOSAPy.mat')
Data = mat73.loadmat('MegaStagePy.mat')
Table = mat73.loadmat('FinalTablePy.mat')

In [4]:
mm = DataOSA['mm']

#Xn = DataOSA['X_alt']
#Xns = DataOSA['Xns']

#y = DataOSA['Yalt']
#y2 = DataOSA['Yalt2']

Xn = Data['Xss']
mm = Data['mm']
y5 = Data['Y5']

y_AHI = DataOSA['y_AHI']
y_ODI = DataOSA['y_ODI']

names_features = ['Mode rawEDA', 'Median rawEDA', 'max(abs) rawEDA', 'Line length rawEDA', '10% quantile rawEDA', '75% quantile rawEDA',
                'SVD entropy rawEDA',
                'Nonlinear energy rawEDA', 'Shannon entropy rawEDA',
                'Mode detEDA', 'Median detEDA', 'max(abs) detEDA', 'Line length detEDA', '10% quantile detEDA', '75% quantile detEDA',
                'SVD entropy detEDA', 'Nonlinear Energy detEDA', 'Shannon entropy detEDA',
                'Mean d(rawEDA)', 'Variance d(rawEDA)', 'Median d(rawEDA)', 'Number above zero d(rawEDA)',
                'Mean d2(rawEDA)', 'Variance d2(rawEDA)', 'Median d2(rawEDA)', 'Number above zero d2(rawEDA)',
                'Mean d(detEDA)', 'Variance d(detEDA)', 'Median d(detEDA)', 'Number above zero d(detEDA)',
                'Mean d2(detEDA)', 'Variance d2(detEDA)', 'Median d2(detEDA)', 'Number above zero d2(detEDA)',
                'Max Periodogram rawEDA', 'Frequency of max PSD rawEDA', 'Fisher g ratio rawEDA',
                'Max Periodogram detEDA', 'Frequency of max PSD detEDA', 'Fisher g ratio detEDA',
                'Max det. coeff. DL 1',
                'Mean det. coeff. DL 1', 'Standard deviation det. coeff. DL 1',
                'Median det. coeff. DL 1', '2-norm det. coeff. DL 1',
                'Normalised det. coeff. above zero DL 1',
                'Max det. coeff. DL 2', 'Mean det. coeff. DL 2',
                'Standard deviation det. coeff. DL 2', 'Median det. coeff. DL 2',
                '2-norm det. coeff. DL 2',
                'Normalised det. coeff. above zero DL 2',
                'Max det. coeff. DL 3', 'Mean det. coeff. DL 3',
                'Standard deviation det. coeff. DL 3', 'Median det. coeff. DL 3',
                '2-norm det. coeff. above zero DL 3',
                'Normalised det. coeff. above zero DL 3',
                'Max det. coeff. DL 4', 'Mean det. coeff. DL 4',
                'Standard deviation det. coeff. DL 4', 'Median det. coeff. DL 4',
                '2-norm det. coeff. DL 4',
                'Normalised det. coeff. above zero DL 4',
                'Lyapunov exponent rawEDA', 'max(upper envelope) rawEDA', 'min(lower envelope) rawEDA',
                'Lyapunov exponent detEDA', 'max(upper envelope) detEDA', 'min(lower envelope) detEDA',
                'Sum of correlation between epochs', 'Max convolution value diffEDA',
                'Normalised event samples', 'Normalised event energy', 'Normalised storm samples', 'Normalised storm energy',
                'Sex'
               ]
Xnpd = pd.DataFrame(Xn, columns = names_features)
map_3 = {'Wake' : 0, 'N1' : 1, 'N2' : 1, 'N3' : 1, 'REM' : 2}
map_4 = {'Wake' : 0, 'N1' : 1, 'N2' : 1, 'N3' : 2, 'REM' : 3}
map_5 = {'Wake' : 0, 'N1' : 1, 'N2' : 2, 'N3' : 3, 'REM' : 4}
map_sp = {'Wake' : 0, 'N1' : 1, 'N2' : 2, 'N3' : 3, 'REM' : 0}
n_class = 3

y_general = pd.DataFrame(y5, columns = ['Sleep Stages'])
map_sleep = {0.0 : 'Wake', 1.0 : 'N1', 2.0 : 'N2', 3.0 : 'N3', 4.0 : 'REM'}
y_general['Sleep Stages'] = y_general['Sleep Stages'].map(map_sleep)
ypd = pd.DataFrame(columns = ['Sleep Stages'])
classes_sp = ['W/REM', 'N1', 'N2', 'N3']

if n_class == 5:
    classes = ['Wale', 'N1', 'N2', 'N3', 'REM']
    ypd['Sleep Stages'] = y_general['Sleep Stages'].map(map_5)
elif n_class == 4:
    classes = ['Wake', 'Light', 'Deep', 'REM']
    ypd['Sleep Stages'] = y_general['Sleep Stages'].map(map_4)
elif n_class == 3:
    classes = ['Wake', 'NREM', 'REM']
    ypd['Sleep Stages'] = y_general['Sleep Stages'].map(map_3)

Xypd = pd.concat([Xnpd, ypd], axis = 1)

standard = xgb.XGBClassifier(tree_method = 'hist',
                                     learning_rate = 0.2,
                                     max_depth = 6,
                                     num_class = n_class,
                                     eval_metric = ['mlogloss', 'auc'],
                                     objective = 'multi:softmax',
                                     use_label_encoder = False,
                                     n_estimators = 200,
                                    )

Yahipd = pd.DataFrame(y_AHI, columns = ['OSA'])
Yodipd = pd.DataFrame(y_ODI, columns = ['OSA'])

classes_OSA = ['Healthy', 'Mild OSA', 'Severe OSA']
    
Xypd_ODI = pd.concat([Xnpd, Yodipd], axis = 1)
Xypd_AHI = pd.concat([Xnpd, Yahipd], axis = 1)
Diff = Table['Diff']
AHI = Table['AHI']
ODI = Table['ODI']
n_classOSA = 3
#classes_OSA = ['Non-OSA', 'OSA']

yODI2 = pd.DataFrame(y_ODI, columns = ['OSA'])
map2 = {0.0 : 0, 1.0 : 1, 2.0 : 1}
yODI2['OSA'] = yODI2['OSA'].map(map2)

yAHI2 = pd.DataFrame(y_AHI, columns = ['OSA'])
yAHI2['OSA'] = yAHI2['OSA'].map(map2)

Xypd_ODI2 = pd.concat([Xnpd, yODI2], axis = 1)
Xypd_AHI2 = pd.concat([Xnpd, yAHI2], axis = 1)

In [6]:
cor_matrix = Xnpd.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k = 1).astype(bool))
upper_tri['10% quantile detEDA']

Mode rawEDA                 0.709764
Median rawEDA               0.692975
max(abs) rawEDA             0.582992
Line length rawEDA          0.015914
10% quantile rawEDA         0.720288
                              ...   
Normalised event samples         NaN
Normalised event energy          NaN
Normalised storm samples         NaN
Normalised storm energy          NaN
Sex                              NaN
Name: 10% quantile detEDA, Length: 77, dtype: float64

In [7]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > .8)]
print(len(to_drop))
print(to_drop)

37
['Median rawEDA', 'max(abs) rawEDA', '10% quantile rawEDA', '75% quantile rawEDA', 'Shannon entropy rawEDA', 'Median detEDA', '10% quantile detEDA', '75% quantile detEDA', 'Shannon entropy detEDA', 'Variance d2(rawEDA)', 'Number above zero d2(rawEDA)', 'Variance d(detEDA)', 'Number above zero d(detEDA)', 'Mean d2(detEDA)', 'Variance d2(detEDA)', 'Number above zero d2(detEDA)', 'Max Periodogram detEDA', 'Max det. coeff. DL 1', 'Mean det. coeff. DL 1', 'Standard deviation det. coeff. DL 1', '2-norm det. coeff. DL 1', 'Max det. coeff. DL 2', 'Mean det. coeff. DL 2', 'Standard deviation det. coeff. DL 2', '2-norm det. coeff. DL 2', 'Max det. coeff. DL 3', 'Mean det. coeff. DL 3', 'Standard deviation det. coeff. DL 3', '2-norm det. coeff. above zero DL 3', 'Max det. coeff. DL 4', 'Mean det. coeff. DL 4', 'Standard deviation det. coeff. DL 4', '2-norm det. coeff. DL 4', 'max(upper envelope) rawEDA', 'min(lower envelope) rawEDA', 'max(upper envelope) detEDA', 'min(lower envelope) detEDA']


In [78]:
Xysrpd = Xypd.drop(columns = to_drop, axis = 1)
XysrODI = Xypd_ODI.drop(columns = to_drop, axis = 1)
XysrAHI = Xypd_AHI.drop(columns = to_drop, axis = 1)
XysrAHI2 = Xypd_AHI2.drop(columns = to_drop, axis = 1)
XysrODI2 = Xypd_ODI2.drop(columns = to_drop, axis = 1)
number_reduced = XysrODI.shape[1] - 1
print(number_reduced)
#XysrODI2
print(XysrODI.columns)
#sleep = XysrODI.columns
osa = XysrODI.columns

40
Index(['Mode rawEDA', 'Line length rawEDA', 'SVD entropy rawEDA',
       'Nonlinear energy rawEDA', 'Mode detEDA', 'max(abs) detEDA',
       'Line length detEDA', 'SVD entropy detEDA', 'Nonlinear Energy detEDA',
       'Mean d(rawEDA)', 'Variance d(rawEDA)', 'Median d(rawEDA)',
       'Number above zero d(rawEDA)', 'Mean d2(rawEDA)', 'Median d2(rawEDA)',
       'Mean d(detEDA)', 'Median d(detEDA)', 'Median d2(detEDA)',
       'Max Periodogram rawEDA', 'Frequency of max PSD rawEDA',
       'Fisher g ratio rawEDA', 'Frequency of max PSD detEDA',
       'Fisher g ratio detEDA', 'Median det. coeff. DL 1',
       'Normalised det. coeff. above zero DL 1', 'Median det. coeff. DL 2',
       'Normalised det. coeff. above zero DL 2', 'Median det. coeff. DL 3',
       'Normalised det. coeff. above zero DL 3', 'Median det. coeff. DL 4',
       'Normalised det. coeff. above zero DL 4', 'Lyapunov exponent rawEDA',
       'Lyapunov exponent detEDA', 'Sum of correlation between epochs',
       'Max

In [80]:
CM_test = np.zeros((n_class, n_class))
CM_test2 = np.zeros((n_class, n_class))

features_summary = np.zeros((len(mm), number_reduced))
features_summary2 = np.zeros((len(mm), number_reduced))
features_diff = np.zeros((len(mm), number_reduced))
scores = np.zeros((len(mm), 5))
scores2 = np.zeros((len(mm), 5))
train_sizes = np.zeros((len(mm),))
sm = SMOTE(random_state = 42)
train_size_increased = 0.15

scores_xgb = np.zeros((len(mm), 8))
mat_predictions_xgb_ODI = np.zeros((len(mm), len(classes_OSA)))
features_xgb_train_ODI = np.zeros((len(mm), number_reduced))
mat_predictions_xgb_AHI = np.zeros((len(mm), len(classes_OSA)))
features_xgb_train_AHI = np.zeros((len(mm), number_reduced))
CM_AHI = np.zeros((len(classes_OSA), len(classes_OSA)))
CM_ODI = np.zeros((len(classes_OSA), len(classes_OSA)))
AUC_AHI = np.zeros((len(mm), 3))
AUC_ODI = np.zeros((len(mm), 3))
predictions_AHI = np.array([])
predictions_ODI = np.array([])
true_AHI = np.array([])
true_ODI = np.array([])

for k in range(len(mm)):
    print(k + 1)
    start = time.time()
    #test_k, train_k = IntraIndividualBuilder(Xysrpd, mm, k)
    
    #test_train_k, test_test_k = train_test_split(test_k, train_size = 0.25, random_state = 42)

    #Xtrain_sm, Ytrain_sm = sm.fit_resample(train_k.drop(columns = ['Sleep Stages']), train_k.iloc[:, -1])
    #Xtrain2_sm, Ytrain2_sm = sm.fit_resample(test_train_k.drop(columns = ['Sleep Stages']), test_train_k.iloc[:, -1])
    
    #Mdl_k = clone(standard)
    #Mdl_k.fit(Xtrain_sm, Ytrain_sm)
    #predictions_test = Mdl_k.predict(test_k.drop(columns = ['Sleep Stages']))
    
    #Mdl_k2 = clone(Mdl_k)
    

    #train2, test2 = First2Hours(test_k)    
    
    #Mdl_k2.fit(Xtrain2_sm, Ytrain2_sm)
    #predictions_test2 = Mdl_k2.predict(test_test_k.drop(columns = ['Sleep Stages']))
    #CM_test2 += confusion_matrix(test_test_k.iloc[:, -1], predictions_test2)
    
    #Mdl_k2.fit(train2.drop(columns = ['Sleep Stages']), train2.iloc[:, -1])
    #predictions_test2 = Mdl_k2.predict(test2.drop(columns = ['Sleep Stages']))
   
    
    #CM_test += confusion_matrix(test_k.iloc[:, -1], predictions_test)
    #CM_test2 += confusion_matrix(test2.iloc[:, -1], predictions_test2)
    
    
    #scores[k, 0] = f1_score(test_k.iloc[:, -1], predictions_test, average = 'macro')
    #scores[k, 1] = f1_score(test_k.iloc[:, -1], predictions_test, average = 'micro')
    #scores[k, 2] = cohen_kappa_score(test_k.iloc[:, -1], predictions_test)
    #scores[k, 3] = recall_score(test_k.iloc[:, -1], predictions_test, average = 'macro')
    #scores[k, 4] = recall_score(test_k.iloc[:, -1], predictions_test, average = 'micro')
    
    #scores2[k, 0] = f1_score(test_test_k.iloc[:, -1], predictions_test2, average = 'macro')
    #scores2[k, 1] = f1_score(test_test_k.iloc[:, -1], predictions_test2, average = 'micro')
    #scores2[k, 2] = cohen_kappa_score(test_test_k.iloc[:, -1], predictions_test2)
    #scores2[k, 3] = recall_score(test_test_k.iloc[:, -1], predictions_test2, average = 'macro')
    #scores2[k, 0] = f1_score(test2.iloc[:, -1], predictions_test2, average = 'macro')
    #scores2[k, 1] = f1_score(test2.iloc[:, -1], predictions_test2, average = 'micro')
    #scores2[k, 2] = cohen_kappa_score(test2.iloc[:, -1], predictions_test2)
    #scores2[k, 3] = recall_score(test2.iloc[:, -1], predictions_test2, average = 'macro')

    
    #features_summary[k, :] = Mdl_k.feature_importances_
    #features_summary2[k, :] = Mdl_k2.feature_importances_
    #features_diff[k, :] = Mdl_k.feature_importances_ - Mdl_k2.feature_importances_

    #######################
    
    test_AHI, train_AHI = IntraIndividualBuilder(XysrAHI, mm, k)
    test_ODI, train_ODI = IntraIndividualBuilder(XysrODI, mm, k)
            
    Mdl_xgb_AHI = xgb.XGBClassifier(tree_method = 'hist',
                                learning_rate = 0.01,
                                #objective = 'binary:logistic',
                                objective = 'multi:softmax',
                                use_label_encoder = False,
                                verbosity = 0,
                                max_depth = 6,
                                n_estimators = 200,
                               )
    Mdl_xgb_ODI = xgb.XGBClassifier(tree_method = 'hist',
                                learning_rate = 0.01,
                                #objective = 'binary:logistic',
                                objective = 'multi:softmax',
                                use_label_encoder = False,
                                verbosity = 0,
                                max_depth = 6,
                                n_estimators = 200,
                               )
    
    Xtrain_ODI = train_ODI.drop(columns = ['OSA'])
    Ytrain_ODI = train_ODI.iloc[:, -1]
    Xtrain_AHI = train_AHI.drop(columns = ['OSA'])
    Ytrain_AHI = train_AHI.iloc[:, -1]
    
    Xtrain_ODIsm, Ytrain_ODIsm = sm.fit_resample(Xtrain_ODI, Ytrain_ODI)
    Xtrain_AHIsm, Ytrain_AHIsm = sm.fit_resample(Xtrain_AHI, Ytrain_AHI)
    
    Xtest_ODI = test_ODI.drop(columns = ['OSA'])
    Ytest_ODI = test_ODI.iloc[:, -1]
    Xtest_AHI = test_AHI.drop(columns = ['OSA'])
    Ytest_AHI = test_AHI.iloc[:, -1]
    
    #Mdl_xgb_ODI.fit(Xtrain_ODI, Ytrain_ODI)
    #Mdl_xgb_AHI.fit(Xtrain_AHI, Ytrain_AHI)
    
    Mdl_xgb_ODI.fit(Xtrain_ODIsm, Ytrain_ODIsm)
    Mdl_xgb_AHI.fit(Xtrain_AHIsm, Ytrain_AHIsm)
    
    predictions_xgb_ODI = Mdl_xgb_ODI.predict(Xtest_ODI)
    predictions_xgb_AHI = Mdl_xgb_AHI.predict(Xtest_AHI)
    
    mat_predictions_xgb_AHI[k, :] = Evaluation(predictions_xgb_AHI, 3)
    mat_predictions_xgb_ODI[k, :] = Evaluation(predictions_xgb_ODI, 3)
        
    xgb_osa_eval_AHI = OSAEvaluation(predictions_xgb_AHI, Ytest_AHI.values)
    xgb_osa_eval_ODI = OSAEvaluation(predictions_xgb_ODI, Ytest_ODI.values)

    scores_xgb[k, 0] = accuracy_score(Ytest_AHI, predictions_xgb_AHI)
    scores_xgb[k, 1] = f1_score(Ytest_AHI, predictions_xgb_AHI, labels = np.unique(predictions_xgb_AHI), average = 'macro')
    scores_xgb[k, 2] = sum(xgb_osa_eval_AHI) / len(xgb_osa_eval_AHI)
    scores_xgb[k, 3] = accuracy_score(Ytest_ODI, predictions_xgb_ODI)
    scores_xgb[k, 4] = f1_score(Ytest_ODI, predictions_xgb_ODI, labels = np.unique(predictions_xgb_ODI), average = 'macro')
    scores_xgb[k, 5] = sum(xgb_osa_eval_ODI) / len(xgb_osa_eval_ODI)
    #scores_xgb[k, 6] = recall_score(Ytest_AHI, predictions_xgb_AHI, labels = np.arange(n_class), average = 'macro')
    #scores_xgb[k, 7] = recall_score(Ytest_ODI, predictions_xgb_ODI, labels = np.arange(n_class), average = 'macro')
    
    predictions_AHI = np.concatenate([predictions_AHI, predictions_xgb_AHI])
    predictions_ODI = np.concatenate([predictions_ODI, predictions_xgb_ODI])
    true_AHI = np.concatenate([true_AHI, Ytest_AHI])
    true_ODI = np.concatenate([true_ODI, Ytest_ODI])
    
    features_xgb_train_AHI[k, :] = Mdl_xgb_AHI.feature_importances_
    features_xgb_train_ODI[k, :] = Mdl_xgb_ODI.feature_importances_

    #CM_AHI += confusion_matrix(Ytest_AHI, predictions_xgb_AHI, labels = np.arange(n_classOSA))
    #CM_ODI += confusion_matrix(Ytest_ODI, predictions_xgb_ODI, labels = np.arange(n_classOSA))
    
    #    AUC_AHI[i, :] = roc_auc_score(Ytest_AHI, predictions_xgb_AHI)
    #    AUC_ODI[i, :] = roc_auc_score(Ytest_ODI, predictions_xgb_ODI)
    
    finish = time.time()

    #del Mdl_xgb_AHI, Mdl_xgb_ODI
    print(finish - start)
    

1
20.808088064193726
2
20.18749213218689
3
20.804983139038086
4
20.91813063621521
5
20.831027507781982
6
21.157108545303345
7
21.304754972457886
8
21.12535548210144
9
20.818381309509277
10
21.923776388168335
11
20.919044971466064
12
21.16933250427246
13
21.76720929145813
14
21.448274612426758
15
21.556912422180176
16
20.901105165481567
17
20.75359797477722
18
20.463568449020386
19
21.407569408416748
20
20.655099391937256
21
21.304754734039307
22
23.06016731262207
23
21.283825159072876
24
20.723698139190674
25
20.878410816192627
26
22.104082107543945
27
22.496932983398438
28
21.28482174873352
29
21.478320121765137
30
20.564749717712402
31
21.048611402511597
32
21.932834148406982
33
21.531610012054443
34
20.927137851715088
35
21.171722173690796
36
21.35125970840454
37
21.695448875427246
38
21.49910593032837
39
21.66460418701172
40
21.37259793281555
41
21.45036792755127
42
21.537975549697876
43
21.081501007080078
44
21.140305995941162
45
21.329671621322632
46
21.69544792175293
47
21.42535

In [81]:
scores_pdxgb = pd.DataFrame(scores_xgb, columns = ['Accuracy AHI', 'F1-Macro AHI', 'AHI OSA-Specific', 
                                                'Accuracy ODI', 'F1-Macro ODI', 'ODI OSA-Specific', 
                                                'Recall AHI', 'Recall ODI'])
print('Mean Accuracy AHI: ', np.mean(scores_pdxgb['Accuracy AHI']))
print('Mean F1 AHI: ', np.mean(scores_pdxgb['F1-Macro AHI']))
print('Mean F1 AHI: ', np.mean(scores_pdxgb['AHI OSA-Specific']))

print('Mean Accuracy ODI: ', np.mean(scores_pdxgb['Accuracy ODI']))
print('Mean F1 ODI: ', np.mean(scores_pdxgb['F1-Macro ODI']))
print('Mean F1 ODI: ', np.mean(scores_pdxgb['ODI OSA-Specific']))


Mean Accuracy AHI:  0.5512040111842544
Mean F1 AHI:  0.3188367318695015
Mean F1 AHI:  0.7515206342006021
Mean Accuracy ODI:  0.5402461202270822
Mean F1 ODI:  0.3347460032121691
Mean F1 ODI:  0.821259661426896


In [138]:
scores_pdxgb = pd.DataFrame(scores_xgb, columns = ['Accuracy AHI', 'F1-Macro AHI', 'AHI OSA-Specific', 
                                                'Accuracy ODI', 'F1-Macro ODI', 'ODI OSA-Specific', 
                                                'Recall AHI', 'Recall ODI'])
print('Mean Accuracy AHI: ', np.mean(scores_pdxgb['Accuracy AHI']))
print('Mean F1 AHI: ', np.mean(scores_pdxgb['F1-Macro AHI']))
print('Mean F1 AHI: ', np.mean(scores_pdxgb['AHI OSA-Specific']))

print('Mean Accuracy ODI: ', np.mean(scores_pdxgb['Accuracy ODI']))
print('Mean F1 ODI: ', np.mean(scores_pdxgb['F1-Macro ODI']))
print('Mean F1 ODI: ', np.mean(scores_pdxgb['ODI OSA-Specific']))


Mean Accuracy AHI:  0.5374912650964238
Mean F1 AHI:  0.2603764160073899
Mean F1 AHI:  0.8185263591944395
Mean Accuracy ODI:  0.6293998837848069
Mean F1 ODI:  0.3294670082613276
Mean F1 ODI:  0.8433539873813231


In [11]:
CM_test_sum = np.sum(CM_test, axis = 1, keepdims = True)
CM_test_norm = CM_test.astype('float') / CM_test.sum(axis = 1)[:, np.newaxis]

#CM_AHI_sum = np.sum(CM_ODI, axis = 1, keepdims = True)
#CM_AHI_norm = CM_ODI.astype('float') / CM_ODI.sum(axis = 1)[:, np.newaxis]

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['xtick.labelsize'] = 28
plt.rcParams['ytick.labelsize'] = 28

fig, ax = plt.subplots(figsize = (12, 10), dpi = 300)

sns.set(rc={'text.usetex': True})
sns.heatmap(CM_test_norm, 
            annot = True, 
            fmt = '.2f', 
            xticklabels = classes, 
            yticklabels = classes,
            cbar = False,
            annot_kws = {'size' : 32}
           )
plt.setp(ax.get_yticklabels(), rotation = 0, ha = "right",
         rotation_mode = "anchor")
plt.xlabel('Predicted sleep stages', fontsize = 28)
plt.ylabel('True sleep stages', fontsize = 28)
#plt.savefig(r'C:\Users\Jacopo\Desktop\Paper - EDA Sleep Staging\Frontiers Final\Revision\PlotsRevision\CM5_uni.eps', format = 'eps')
plt.show()

In [9]:
CM_test2_sum = np.sum(CM_test2, axis = 1, keepdims = True)
CM_test2_norm = CM_test2.astype('float') / CM_test2.sum(axis = 1)[:, np.newaxis]

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['xtick.labelsize'] = 28
plt.rcParams['ytick.labelsize'] = 28

fig, ax = plt.subplots(figsize = (12, 10), dpi = 300)

sns.set(rc={'text.usetex': True})
sns.heatmap(CM_test2_norm, 
            annot = True, 
            fmt = '.2f', 
            xticklabels = classes, 
            yticklabels = classes,
            cbar = False,
            annot_kws = {'size' : 32}
           )
plt.setp(ax.get_yticklabels(), rotation = 0, ha = "right",
         rotation_mode = "anchor")
plt.xlabel('Predicted sleep stages', fontsize = 28)
plt.ylabel('True sleep stages', fontsize = 28)
#plt.savefig(r'C:\Users\Jacopo\Desktop\Paper - EDA Sleep Staging\Frontiers Final\Revision\PlotsRevision\CM5_pers.eps', format = 'eps')
plt.show()

In [14]:
d = pd.DataFrame(data = Xn,
                 columns = np.arange(1, 1 + Xnpd.shape[1]))
cor_matrix = d.corr().abs()
tri_matrix = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k = 1).astype(bool))
cmap = sns.diverging_palette(230, 20, as_cmap = True)
mask = np.triu(np.ones_like(cor_matrix, dtype=bool))

In [16]:
f, ax = plt.subplots(figsize=(16, 14))

sns.set(rc={'text.usetex': True})
sns.set_theme(style="white")
sns.heatmap(cor_matrix, 
            xticklabels = cor_matrix.columns, 
            yticklabels=  cor_matrix.columns,
            mask = mask, 
            cmap = cmap,  
            center = 0,
            square = True, 
            linewidths = .01, 
            #cbar = False,
            cbar_kws = {"shrink": 0.5}
           )
plt.setp(ax.get_yticklabels(), rotation = 0, ha = "right",
         rotation_mode = "anchor")
plt.tight_layout()

In [9]:
dd = d.corr().abs()
upper_tri = dd.where(np.triu(np.ones(dd.shape), k = 1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > .8)]
print(len(to_drop))
print(to_drop)

37
[2, 3, 5, 6, 9, 11, 14, 15, 18, 24, 26, 28, 30, 31, 32, 34, 38, 41, 42, 43, 45, 47, 48, 49, 51, 53, 54, 55, 57, 59, 60, 61, 63, 66, 67, 69, 70]


In [10]:
Xysrpd = d.drop(columns = to_drop, axis = 1)
cor = Xysrpd.corr()
tri_matrix = cor.where(np.triu(np.ones(cor.shape), k = 1).astype(bool))
cmap = sns.diverging_palette(230, 20, as_cmap = True)
mask2 = np.triu(np.ones_like(cor, dtype=bool))

In [18]:
f, ax = plt.subplots(figsize=(16, 14))

sns.set(rc={'text.usetex': True})
sns.set_theme(style="white")
sns.heatmap(cor, 
            xticklabels = cor.columns, 
            yticklabels=  cor.columns, 
            mask = mask2, 
            cmap = cmap,  
            center = 0,
            square = True, 
            linewidths = .01, 
            #cbar = False,
            cbar_kws = {"shrink": 0.5}
           )
plt.setp(ax.get_yticklabels(), rotation = 0, ha = "right",
         rotation_mode = "anchor")
plt.tight_layout()